In [52]:

from libdnn import dnn
import numpy as np
from scipy import signal
from time import time

In [53]:
tensor = np.array([[
    [[1,17], [5,21], [9,25], [13,29]],
    [[2,18], [6,22], [10,26], [14,30]],
    [[3,19], [7,23], [11,27], [15,31]],
    [[4,20], [8,24], [12,28], [16,32]]
    ]
])
print(tensor.shape)

(1, 4, 4, 2)


In [54]:
filter_t = np.array([
    [[[0,1],[1,2]], [[1,2],[0,1]], [[0,1],[1,2]]],
    [[[1,2],[0,1]], [[1,2],[0,1]], [[1,2],[0,1]]],
    [[[0,1],[1,2]], [[1,2],[0,1]], [[0,1],[1,2]]]
])
print(filter_t.shape)

(3, 3, 2, 2)


In [55]:
def pad(x, pd_n):
    for i in range(pd_n):
        x = np.insert(x, 0, [[0]*x.shape[-1]], axis=1)
        x = np.insert(x, x.shape[1], [[0]*x.shape[-1]], axis=1)
        #x = np.roll(x, -2)
    
    for i in range(pd_n):
        x = np.insert(x, 0, [[[0]*x.shape[-1]]*x.shape[1]], axis=0)
        x = np.insert(x, x.shape[0], [[[0]*x.shape[-1]]*x.shape[1]], axis=0)
    return x

def __conv2d(x2d, f2d, strides):
    #print(x2d)
    #print(f2d)
    fw, fh = f2d.shape[:2]
    op = 0
    for x in range(fw):
        for y in range(fh):
            op += x2d[x,y]*f2d[x,y]
    #print(op)
    return op



In [59]:
def np4d_to_list(np_arr):
    ns = np_arr.shape
    nl = []
    for oc in range(ns[3]):
        nl.append(list())
        for ic in range(ns[2]):
            nl[-1].append(list())
            for w in range(ns[1]):
                nl[-1][-1].append(list())
                for h in range(ns[0]):
                    nl[-1][-1][-1].append(np_arr[h][w][ic][oc])
    return nl

def np3d_to_list(np_arr):
    ns = np_arr.shape
    nl = []
    for oc in range(ns[2]):
        nl.append(list())
        for w in range(ns[1]):
            nl[-1].append(list())
            for h in range(ns[0]):
                nl[-1][-1].append(np_arr[h][w][oc])
    return nl


def convolve2d(x, cnv_filter, strides, padding):
    fls = cnv_filter.shape
    if padding == 'same':
        x = pad(x, fls[0]//2)
    
    xsp = x.shape
    #print(fls)
    
    y = np.zeros((((xsp[0]-fls[0])//strides[0])+1,((xsp[1]-fls[1])//strides[1])+1, fls[3]))
    for zf in range(fls[3]):
        curr_filter = cnv_filter[:,:,:,zf]
        cfs = curr_filter.shape
        #tmp = np.zeros((((xsp[0]-fls[0])//strides[0])+1,((xsp[1]-fls[1])//strides[1])+1, fls[2]))
        for xi in range(0, y.shape[1], strides[1]):
            for yi in range(0, y.shape[0], strides[0]):
                for zi in range(fls[2]):
                    inp_x = x[yi:yi+fls[0],xi:xi+fls[1],zi]
                    inp_f = curr_filter[:,:,zi]
                    y[yi,xi,zf] = y[yi,xi,zf] + (__conv2d(np.transpose(inp_x), inp_f, strides)).sum()
    return y

for inp_t in tensor:
    t0 = time()
    cnv_res = convolve2d(inp_t, filter_t, (1,1), 'valid')
    t1 = time()
    print(np3d_to_list(cnv_res))
    print("time taken:", t1-t0)
t0 = time()
cnv_res = dnn.conv2d([np3d_to_list(tensor[0])], np4d_to_list(filter_t), (1,1),'valid')
t1 = time()
print(cnv_res)
print("time taken:", t1-t0)

[[[118.0, 127.0], [154.0, 163.0]], [[370.0, 397.0], [478.0, 505.0]]]
time taken: 0.00015664100646972656
[[[[118, 127], [154, 163]], [[370, 397], [478, 505]]]]
time taken: 0.00019025802612304688


In [57]:
v1 = np.array([
    []
])

v2 = np.array([
    
])
op = np.array([v1, v2])
print(op)


[array([], shape=(1, 0), dtype=float64) array([], dtype=float64)]


In [58]:

print(dnn.flatten([np3d_to_list(tensor[0])]))

[[[1, 17, 2, 18, 3, 19, 4, 20, 5, 21, 6, 22, 7, 23, 8, 24, 9, 25, 10, 26, 11, 27, 12, 28, 13, 29, 14, 30, 15, 31, 16, 32]]]
